# Load and clean data with Python and pandas

This notebook shows you how to load newspaper text data into a pandas DataFrame, clean the data and extract text from a single newspaper into a text file.

First, we import the pandas library.

In [ ]:
import pandas as pd

## Load data
If we have dowloaded the data to our computer, we define the path to a local data file. Otherwise, we can use a URL from the [LOAR API](https://datasprint2020.kb.dk/framework/#data).

In [ ]:
local_data = 'data/artikler_1848.csv'

In [ ]:
# URL for 1848 newspapers
api_url = 'https://loar.kb.dk/rest/bitstreams/f2437eca-c354-46c0-aee2-022279050ce3/retrieve'

In this case, we use the API and load the data using pandas' `read_csv()` method.

Technical note: `\` needs to be used as an escape character for the data to load correctly. Because `\` in itself is an escape character, which ignores the following character, we need to escape the escape character by using an additional backslash.

In [ ]:
df = pd.read_csv(api_url, escapechar='\\')

Now our data is loaded into a pandas DataFrame. We can inspect the shape of the DataFrame to check that the data are loaded correctly.

In [ ]:
df.shape

We can use `columns` to list the columns of the DataFrame.

In [ ]:
df.columns

The `head()` command returns the first 5 rows of the DataFrame. Here we get a peak of the structure of our data

In [ ]:
df.head()

The `recordID` column is a unique identifier, which we most likely wont use. The `sort_year_asc` column shows the date the newspaper was published. The `editionID` column has information about the newspaper edition and `newspaper_page` tells us on which page the text appeared. Finally, we have the `fulltext_org` column. This is where the actual text from the newspapers is found. The text is split into somewhat arbitrary sections but we don't have to worry about that for now.

## Clean up
Before we start working with the data, we can process it and clean it in different ways. This will make the data much more managable in our future analyses.

### Extracting newspaper title
The `editionId` column contains various metadata. If we split the `editionId` column, we can extract the newspaper title and store it in a separate column.

The title we extract also functions as an identifier that can be used to search for specific newspapers in [Mediestream](http://www2.statsbiblioteket.dk/mediestream/) or [Smurf](http://labs.statsbiblioteket.dk/smurf/). For this reason, we don't modify the title further by adding spaces or removing year of establishment. However, if you want cleaner title values, you are welcome to make adjustments.

In [ ]:
df['title'] = df['editionId'].str.split(expand=True)[0]

The DataFrame now has a `title` column.

In [ ]:
df.head()

### Filtering the data
When cleaning text data, we can use regular expression as a sophisticated Find & Replace tool. Regular expressions are included with pandas. If needed outside pandas, we can use Python's built-in `re` library.

Regular expressions are notoriously confusing and will often cause more problems than they solve. However, when used carefully, they are a very powerful tool.

Below we access the newspaper text in the `fulltext_org` column of our DataFrame. We use the pandas string method `replace()`. The first argument is what we want to replace (in this case a regular expression). The second argument is what we want to replace it with (in this case nothing because we just want to remove characters).

Briefly, this regular expression removes all punctuation and symbols caused by faulty OCR. The expression `[^\w\d\s]` finds all letters (`\w`), digits (`\d`) and spaces (`\s`). The square brackets indicate that any of the elements within will be replaced. Finally, the caret (`^`) negates the entire expression meaning that characters that are not a letter, a digit or a space will be replaced.

In [ ]:
df['fulltext_org'].str.replace('[^\w\d\s]', '')

By default, `\w` includes all letters from A-Z, both upper and lower case. If we want to keep other letters specific to our domain, we can add them to the expression.

In [ ]:
df['fulltext_org'].str.replace('[^\w\d\sÆæØøÅåÄäÖöÜü]', '')

Depending on the analysis, we might choose to keep some basic punctuation such as commas and full stops, in order to keep some readability for humans.

In [ ]:
df['fulltext_org'].str.replace('[^\w\d\sÆæØøÅåÄäÖöÜü.,]', '')

Once we are happy with our expression, we save the filtered data by assigning it to the `fulltext_org` column; or a new column if we want to keep the original data.

In [ ]:
df['fulltext_org'] = df['fulltext_org'].str.replace('[^\w\d\sÆæØøÅåÄäÖöÜü.,]', '')

Did I mention that regular expression can be confusing? If you want to learn more about regular expressions have a look at the [documentation](https://docs.python.org/3/howto/regex.html).

### Case conversion
Another way to make the data more managable is to convert all letters to lower case. This is useful if we want to search for specific terms and when we start counting occurences of terms because we wont have to worry about how the words appear in the original text.

To convert the text data in the `fulltext_org` column to lower case we simply use the string method `lower()`.

In [ ]:
df['fulltext_org'] = df['fulltext_org'].str.lower()

## Single newspaper text extraction
Once we have cleaned our text, we might want to select parts of the data to work with. We use the following method to extract all text of a specific newspaper from the DataFrame into one string of text.

First, we use the `unique()` method to get a list of available newspaper title IDs.

In [ ]:
df['title'].unique()

We select a newspaper and use the string method `join()` to combine the text fields of all rows where the title field matches the selected newspaper.

In [ ]:
newspaper = 'dannevirke1838'

full_text = ' '.join(df[df['title'] == newspaper]['fulltext_org'])

Optionally, we can save the full text string to a text file for easier future access.

In [ ]:
file_destination = f'{newspaper}_fulltext.txt'

with open(file_destination, 'w', encoding='utf-8') as f:
    f.write(full_text)

## Conclusion

This notebook briefly demonstrated how we can handle and process our data with Python and pandas. These basic tools are useful starting points when we begin analysing our data and you are encouraged to revisit them later on.